# Manage Quota

Note: 
- Geo Engine has to have "pro" feature enabled
- quota_check has to be enabled in Settings
- admin_token has to be set in Settings

In [13]:
import geoengine as ge
from geoengine import Colorizer

from datetime import datetime

# Select a user

In [14]:
email = "foo@example.com"
password = "secret123"

# register the user, if it doesn't exist yet
import requests as re
re.post("http://localhost:3030/api/user", json={"email": email, "password": password, "realName": "Foo Bar"})

<Response [409]>

## Initialize as user

In [15]:
ge.initialize("http://localhost:3030/api", credentials=(email, password))

user_id = ge.get_session().user_id

## Access own quota

In [16]:
ge.get_quota()

{'available': 1000, 'used': 1}

## Try to run a query (fails because quota is exhausted)

In [17]:
ports = ge.register_workflow({
    "type": "Vector",
    "operator": {
        "type": "OgrSource",
        "params": {
            "data": {
                "type": "internal",
                "datasetId": "a9623a5b-b6c5-404b-bc5a-313ff72e4e75"
            },
            "attributeProjection": None
        }
    }
})

time = datetime.strptime(
    '2014-04-01T12:00:00.000Z', "%Y-%m-%dT%H:%M:%S.%f%z")

try:
    df = ports.get_dataframe(
        ge.QueryRectangle(
            ge.BoundingBox2D(-180.0, -90.0, 180.0, 90.0),
            ge.TimeInterval(time, time),
            ge.SpatialResolution(0.1, 0.1)
            )
    )
except ge.GeoEngineException as e:
    print(e)

## Initialize Geo Engine as Admin

In [18]:
ge.initialize("http://localhost:3030/api", admin_token="8aca8875-425a-4ef1-8ee6-cdfc62dd7525")

## Access user quota

In [19]:
ge.get_quota(user_id)

{'available': 1000, 'used': 2}

## Update user quota

In [20]:
ge.update_quota(user_id, 1000)

## Verify quota update worked

In [21]:
ge.get_quota(user_id)

{'available': 1000, 'used': 2}

# Go back to the regular user

In [22]:
ge.initialize("http://localhost:3030/api", credentials=(email, password))

ge.get_quota()

{'available': 1000, 'used': 2}

# Rerun the workflow, works now

In [23]:

df = ports.get_dataframe(
        ge.QueryRectangle(
            ge.BoundingBox2D(-180.0, -90.0, 180.0, 90.0),
            ge.TimeInterval(time, time),
            ge.SpatialResolution(0.1, 0.1)
            )
    )

df.head()

,geometry,featurecla,name,natlscale,scalerank,website,start,end
0,POINT (-69.92356 12.43750),Port,Sint Nicolaas,5.0,8,www.rocargo.com/SanNicolas.html,NaT,NaT
1,POINT (-58.95141 -34.15333),Port,Campana,5.0,8,www.consejoportuario.com.ar,NaT,NaT
2,POINT (-59.00495 -34.09889),Port,Zarate,5.0,8,www.consejoportuario.com.ar,NaT,NaT
3,POINT (-62.10088 -38.89444),Port,Puerto Belgrano/Bahia Blanca,5.0,8,None,NaT,NaT
4,POINT (-62.30053 -38.78306),Port,Puerto Galvan/Bahia Blanca,5.0,8,None,NaT,NaT
